In [2]:
'''Plan:
0. Load libraries, 
1. load data.
2. Preliminary EDA.
3. Dealing with missing values, merging train and test.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. [opt] Feature importance, error analysis.
10. Predictions.
'''

# aside:
# when coding for interview ML purposes or Kaggle, never drop any obervations!
# you will have to make predictions for all obs in test sample.

# correct way to deal with missing obs and merge train and test samples:
# 1. Load both samples.
# 2. Impute missing values in both samples, using train sample to impute missing values.
# 3. Concatentate them into df.


### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [3]:
### 1. Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)

test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

train = train.sample(n=200000)

(957919, 120)


In [4]:
### 2. pEDA ###

train.shape
train.describe()

# are there numerical features?

un_colval = pd.DataFrame([[x,len(train[x].unique())] for x in train.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [5]:
### 3. Missing values ###

# given huge difference in accuracy with and without missing values, i suggest creating dummies for them.
# to write more general code, will have to create union of mis_cols for train and test.

train_mis_cols = [col for col in train.columns if train[col].isnull().any()]
test_mis_cols = [col for col in test.columns if test[col].isnull().any()]

for col in train_mis_cols:
    train[col + '_miss'] = (train[col].isnull()).astype(int)

for col in test_mis_cols:
    test[col + '_miss'] = (test[col].isnull()).astype(int)

In [6]:
#train.describe()
#test.describe()

# imputing missing values #

colmnames = train.columns
test['claim'] = np.nan
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train)
train = pd.DataFrame(imp.transform(train))
test = pd.DataFrame(imp.transform(test))
#train.describe()

In [7]:
train['sample'] = 'train'
test['sample'] = 'pred'
df = pd.concat([train, test])
#df.colnames = list(colmnames)+'sample'
df.reset_index(drop=True, inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,sample
0,783049.0,0.097779,0.012276,2614.40,0.192540,0.423960,-0.855970,2772.90,925680.0,-5.830800e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,738585.0,0.001533,0.354130,6242.10,0.303880,0.205060,-7.032700,279.29,1006700.0,2.784100e+14,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train
2,386867.0,0.022685,0.469030,21040.00,0.584090,0.001791,5.324300,1869.00,962440.0,4.917800e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,6056.0,0.091557,0.410210,1707.50,0.394460,0.303500,-0.446710,338.22,24767.0,5.823200e+13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,389357.0,0.090905,0.332530,4543.60,0.000301,0.302340,-7.389900,2214.70,63239.0,4.761200e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.414150,-1.068500,651.22,985000.0,6.079700e+15,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,pred
693470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.355400,-0.870200,2514.20,18004.0,6.073500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
693471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.288030,-1.407100,434.03,333050.0,2.351000e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
693472,1451391.0,0.168000,0.351760,454.79,0.164580,0.169830,0.323850,2331.20,223980.0,-2.795300e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred


In [8]:
newcolnames = list(colmnames) + ['sample']
df.columns = newcolnames
df

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110_miss,f111_miss,f112_miss,f113_miss,f114_miss,f115_miss,f116_miss,f117_miss,f118_miss,sample
0,783049.0,0.097779,0.012276,2614.40,0.192540,0.423960,-0.855970,2772.90,925680.0,-5.830800e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,738585.0,0.001533,0.354130,6242.10,0.303880,0.205060,-7.032700,279.29,1006700.0,2.784100e+14,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train
2,386867.0,0.022685,0.469030,21040.00,0.584090,0.001791,5.324300,1869.00,962440.0,4.917800e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,6056.0,0.091557,0.410210,1707.50,0.394460,0.303500,-0.446710,338.22,24767.0,5.823200e+13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,389357.0,0.090905,0.332530,4543.60,0.000301,0.302340,-7.389900,2214.70,63239.0,4.761200e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.414150,-1.068500,651.22,985000.0,6.079700e+15,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,pred
693470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.355400,-0.870200,2514.20,18004.0,6.073500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
693471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.288030,-1.407100,434.03,333050.0,2.351000e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
693472,1451391.0,0.168000,0.351760,454.79,0.164580,0.169830,0.323850,2331.20,223980.0,-2.795300e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred


In [ ]:
# we have many variables, are all of them useful? lets see correlation

#cors = df.corr()
#cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

In [9]:
### 4. Feature engineering ###

# check skew and possibly transform some variables #

temp = (df.dtypes == np.float64)
num_cols = df.columns[temp]
skew_vals = df[num_cols].skew() 
skew_limit = 1
    
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

print(skew_cols)

                Skew
f118_miss  14.736213
f112_miss   7.794901
f23_miss    7.766490
f104_miss   7.765374
f32_miss    7.763888
...              ...
f13        -1.295179
f110       -1.329500
f58        -1.355262
f46        -1.544203
f91        -1.559867

[187 rows x 1 columns]


In [10]:
### 5. ###

y_train = df.loc[df['sample']=='train',['claim']]
X_train_ = df.drop(columns=['claim'])
X_train = X_train_.loc[X_train_['sample']=='train']
X_pred = X_train_.loc[X_train_['sample']=='pred']

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_pred.shape)

(200000, 1) (200000, 238)
(180000, 1) (180000, 238) (493474, 238)


In [11]:
X_train.drop(columns = ['sample'], inplace=True)
X_test.drop(columns = ['sample'], inplace=True)
X_pred.drop(columns = ['sample'], inplace=True)

In [13]:
X_pred0 = X_pred.copy()

In [14]:
### 6. Scaling ###

ss = StandardScaler()

#for i in X_train.columns[0:119]:
for i in X_train.columns:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109_miss,f110_miss,f111_miss,f112_miss,f113_miss,f114_miss,f115_miss,f116_miss,f117_miss,f118_miss
count,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,...,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05,1.800000e+05
mean,-9.331795e-17,4.515894e-17,-1.260029e-16,-1.500035e-16,9.813385e-17,-5.605393e-17,4.500104e-18,2.448609e-16,1.635235e-16,-1.126802e-16,...,-6.912002e-17,-5.400125e-17,7.397539e-17,-3.667190e-17,2.323080e-17,-1.673723e-17,-1.519772e-17,-3.568504e-17,3.915880e-17,2.561901e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.732446e+00,-5.308969e+00,-2.510885e+00,-2.114201e+00,-1.261839e+00,-2.171144e+00,-5.703571e+00,-1.452595e+00,-1.178510e+00,-1.220329e+00,...,-1.276503e-01,-1.285464e-01,-1.270872e-01,-1.257946e-01,-1.296583e-01,-1.266577e-01,-1.290590e-01,-1.289477e-01,-1.286356e-01,-1.268161e-01
25%,-8.671563e-01,-4.550755e-01,-4.226631e-01,-5.640164e-01,-7.776476e-01,-4.379154e-01,-4.911852e-01,-8.867780e-01,-8.243998e-01,-7.600453e-01,...,-1.276503e-01,-1.285464e-01,-1.270872e-01,-1.257946e-01,-1.296583e-01,-1.266577e-01,-1.290590e-01,-1.289477e-01,-1.286356e-01,-1.268161e-01
50%,-7.061662e-04,1.236807e-03,2.901558e-01,-4.292743e-01,-2.966989e-01,1.546608e-01,-1.446409e-01,-1.361951e-01,-2.506067e-01,-5.541307e-01,...,-1.276503e-01,-1.285464e-01,-1.270872e-01,-1.257946e-01,-1.296583e-01,-1.266577e-01,-1.290590e-01,-1.289477e-01,-1.286356e-01,-1.268161e-01
75%,8.657331e-01,5.769959e-01,7.652422e-01,4.909512e-02,4.422504e-01,7.368135e-01,4.652875e-01,6.778013e-01,5.240277e-01,5.398199e-01,...,-1.276503e-01,-1.285464e-01,-1.270872e-01,-1.257946e-01,-1.296583e-01,-1.266577e-01,-1.290590e-01,-1.289477e-01,-1.286356e-01,-1.268161e-01
max,1.731427e+00,7.204239e+00,1.183533e+00,5.429078e+00,5.198622e+00,1.731285e+00,5.236139e+00,2.953498e+00,4.479547e+00,3.725810e+00,...,7.833905e+00,7.779295e+00,7.868613e+00,7.949466e+00,7.712579e+00,7.895295e+00,7.748396e+00,7.755082e+00,7.773895e+00,7.885434e+00


In [15]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.1, 0.2, 0.5, 1, 2, 3]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)
# after dropping missing I had 86%. while imputing them with median, i had 52%. with miss dummies, 73%.

logistic 0.7344222222222222 {'C': 0.1, 'penalty': 'l2'} 21.83708930015564


In [ ]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

In [ ]:
time1 = time.time()

xgbcl = XGBClassifier(tree_method='gpu_hist', gpu_id=0)

grid_values = {'n_estimators':[100,200,300,400],'eta':[0.05, 0.8, 0.11], 'max_depth':[2,3]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

In [16]:
### 8. performance evaluation ###

yhat_lr = lr.predict(X_test)
#yhat_bt = xgb.predict(X_test)

print('Accuracy of logistic regression is ', 1-(np.abs(yhat_lr-np.array(y_test.claim))).mean())
#print('Accuracy of XGBoost is ', 1-(np.abs(yhat_bt-np.array(y_test.claim))).mean())


Accuracy of logistic regression is  0.73065


In [17]:
### Export results ###
yhat_lr = lr.predict(X_pred).astype(int)
#yhat_bt = xgb.predict(X_pred).astype(int)

submission_df_lr = pd.DataFrame({'id': (X_pred0.id).astype(int), 'claim': yhat_lr}, columns=['id', 'claim'])
#submission_df_bt = pd.DataFrame({'id': (X_pred.id).astype(int), 'claim': yhat_bt}, columns=['id', 'claim'])

submission_df_lr.to_csv('submissions_Sep21_lr1.csv',index=False)
#submission_df_bt.to_csv('submissions_Sep21_bt1.csv',index=False)

In [18]:
os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'submissions_Sep21_lr1.csv')

/kaggle/working/submissions_Sep21_lr1.csv

In [ ]:
# something has gone terribly wrong here. my submissions gte scored as purely random guess...
